In [1]:
import pandas as pd
from nltk.tokenize import RegexpTokenizer
import nltk
regex=nltk.RegexpTokenizer(r'\w+')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [2]:
nltk.download("wordnet")
from nltk.corpus import wordnet #importing wordnet
nltk.download('stopwords')
from nltk.corpus import stopwords    #importing stopwords
stopwords=stopwords.words()   
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\dell\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dell\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\dell\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\dell\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [3]:
df=pd.read_csv('C:/Users/dell/Desktop/Project/train.tsv',encoding = "ISO-8859-1",sep='\t',low_memory=False)

In [4]:
df=df.drop(['Id'],axis=1)

In [5]:
df['EssaySet']=df['EssaySet'].astype('int64')
df['Score1']=df['Score1'].astype('int64')
df['Score2']=df['Score2'].astype('int64')

In [6]:
df.corr(method='pearson')

,EssaySet,Score1,Score2
EssaySet,1.000000,-0.099121,-0.103228
Score1,-0.099121,1.000000,0.912538
Score2,-0.103228,0.912538,1.000000


In [7]:
df['EssayText'] = df['EssayText'].apply(lambda x: " ".join(x.lower() for x in x.split())) ##converting to lowercase

In [8]:
df['EssayText'] = df['EssayText'].str.replace('[^\w\s]','') ##removing punctuatioms

In [9]:
from nltk.corpus import stopwords ##removing stopwords
stop = stopwords.words('english')
df['EssayText'] = df['EssayText'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))

In [10]:
freq = pd.Series(' '.join(df['EssayText']).split()).value_counts()[:10] ##common words removal

In [11]:
freq = list(freq.index)
df['EssayText'] = df['EssayText'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))

In [12]:
freq = pd.Series(' '.join(df['EssayText']).split()).value_counts()[-10:] ##rare words removal


In [13]:
freq = list(freq.index)
df['EssayText'] = df['EssayText'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))

In [14]:
from textblob import TextBlob

In [15]:
df['EssayText'][:5].apply(lambda x: str(TextBlob(x).correct())) ## spell checker

0    additional information need delicate experimen...
1    reading excitement realized additional informa...
2    need trials control set exact amount vinegar p...
3        student list rock better rock worse procedure
4    students able make delicate need tell use much...
Name: EssayText, dtype: object

In [16]:
TextBlob(df['EssayText'][1]).words ##Tokenization

WordList(['reading', 'expirement', 'realized', 'additional', 'information', 'need', 'replicate', 'expireiment', 'one', 'amant', 'vinegar', 'poured', 'container', 'two', 'label', 'containers', 'start', 'yar', 'expirement', 'three', 'write', 'conclusion', 'make', 'sure', 'yar', 'results', 'accurate'])

In [17]:
from nltk.stem import PorterStemmer ##stemming
st = PorterStemmer()
df['EssayText'][:5].apply(lambda x: " ".join([st.stem(word) for word in x.split()]))

0    addit inform need replic experi much vinegar p...
1    read expir realiz addit inform need replic exp...
2    need trial control set exact amount vinegar po...
3          student list rock better rock wors procedur
4    student abl make replic need tell use much vin...
Name: EssayText, dtype: object

In [20]:
from textblob import Word
df['EssayText'] = df['EssayText'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))

In [21]:
df.head()

,EssaySet,Score1,Score2,EssayText
0,1,1,1,additional information need replicate experime...
1,1,1,1,reading expirement realized additional informa...
2,1,1,1,need trial control set exact amount vinegar po...
3,1,0,0,student list rock better rock worse procedure
4,1,2,2,student able make replicate need tell use much...


In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer   ##tf-idf
tfidf = TfidfVectorizer(max_features=1000, lowercase=True, analyzer='word',
 stop_words= 'english',ngram_range=(1,1))
df_vect = tfidf.fit_transform(df['EssayText'])
df_vect

<17207x1000 sparse matrix of type '<class 'numpy.float64'>'
	with 199247 stored elements in Compressed Sparse Row format>

In [23]:
from sklearn.feature_extraction.text import CountVectorizer  ## Bag of Words
bow = CountVectorizer(max_features=1000, lowercase=True, ngram_range=(1,1),analyzer = "word")
df_bow = bow.fit_transform(df['EssayText'])
df_bow

<17207x1000 sparse matrix of type '<class 'numpy.int64'>'
	with 229990 stored elements in Compressed Sparse Row format>